In [23]:
# Imports
import sqlite3
import time
from pyalgdb.execution_tree_creator import ExecTreeCreator
from pyalgdb.top_down import TopDown
from pyalgdb.heaviest_first import HeaviestFirst
from pyalgdb.visualization import Visualization
from pyalgdb.provenance_enhancement import ProvenanceEnhancement 
from pyalgdb.single_stepping import SingleStepping
from pyalgdb.divide_and_query import DivideAndQuery

In [24]:
NOW2_SQLITE_PATH = 'C:/Users/linha/Desktop/ws/py-scripts-examples/min-max/.noworkflow/db.sqlite'

In [25]:
CURSOR = sqlite3.connect(NOW2_SQLITE_PATH).cursor()

In [26]:
creator = ExecTreeCreator(CURSOR)
exec_tree = creator.create_exec_tree()

In [27]:
prov = ProvenanceEnhancement(exec_tree, CURSOR)

In [28]:
dependencies = prov.prov_tools.get_dependencies()
prov.dependencies = dependencies

In [31]:
wrong_data = prov.ask_wrong_data()

Which output data is not correct? 311


In [32]:
wrong_data

In [33]:
prov.enhance(wrong_data)

In [34]:
vis = Visualization(prov.exec_tree)

In [35]:
try:
    vis.view_exec_tree_prov('exec_tree', prov.filtered_dependencies)
except:
    vis.view_exec_tree_prov(str(hash(prov.exec_tree)), prov.filtered_dependencies)

Error: Could not open "exec_tree.gv.pdf" for writing : Invalid argument
